In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, Activation, Add,ZeroPadding2D,\
MaxPooling2D, AveragePooling2D,Flatten
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

In [8]:
class identityBlock(Model):
  def __init__(self,filters,kernelSize):
    super(identityBlock,self).__init__()
    self.cnns = [Conv2D(filters[i], kernelSize[i], padding='same') for i in range(3)]
    self.bns = [BatchNormalization() for i in range(3)]
    self.activation = Activation('relu')
    self.add = Add()
  
  def call(self, inputTensor):
    x = inputTensor
    for i in range(2):
      x = self.cnns[i](x)
      x = self.bns[i](x)
      x = self.activation(x)
    x = self.cnns[2](x)
    x = self.bns[2](x)
    x = self.add([x,inputTensor])
    x = self.activation(x)
    return x

class convolutionalBlock(Model):
  def __init__(self, filters, kernelSize,strides):
    super(convolutionalBlock,self).__init__()
    self.cnns = [Conv2D(filters[i], kernelSize[i], padding='same',strides=strides[i]) for i in range(4)]
    self.bns = [BatchNormalization() for i in range(4)]
    self.activation = Activation('relu')
    self.add = Add()
  
  def call(self, inputTensor):
    br1 = self.cnns[3](inputTensor)
    br1 = self.bns[3](br1)
    br2 = inputTensor
    for i in range(2):
      br2 = self.cnns[i](br2)
      br2 = self.bns[i](br2)
      br2 = self.activation(br2)
    br2 = self.cnns[2](br2)
    br2 = self.bns[2](br2)
    x = self.add([br2, br1])
    x = self.activation(x)
    return x
  

In [9]:
model = identityBlock([3,3,3],[3,3,3])
model.build((None,128, 128, 3))

model.summary()


Model: "identity_block_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           multiple                  84        
_________________________________________________________________
conv2d_14 (Conv2D)           multiple                  84        
_________________________________________________________________
conv2d_15 (Conv2D)           multiple                  84        
_________________________________________________________________
batch_normalization_5 (Batch multiple                  12        
_________________________________________________________________
batch_normalization_6 (Batch multiple                  12        
_________________________________________________________________
batch_normalization_7 (Batch multiple                  12        
_________________________________________________________________
activation_1 (Activation)    multiple             

In [30]:
class resNet50(Model):
  def __init__(self, numberOfClasses):
    super(resNet50,self).__init__()
    self.zeroPadding = ZeroPadding2D((3,3))
    self.conv1 = Conv2D(64,(7,7),strides=(2,2))
    self.bn1 = BatchNormalization()
    self.activation = Activation('relu')
    self.maxPool1 = MaxPooling2D((3,3), strides=(2,2))

    self.cn1 = convolutionalBlock( [64,64,256,256],[1,3,1,1],[1,1,1,1])
    self.id1 = [identityBlock([64,64,256],[1,3,1]) for i in range(2)]

    self.cn2 = convolutionalBlock( [128,128,512,512],[1,3,1,1],[2,1,1,2])
    self.id2 = [identityBlock([128,128,512],[1,3,1]) for i in range(3)]

    self.cn3 = convolutionalBlock([256, 256, 1024,1024],[1,3,1,1],[2,1,1,2])
    self.id3 = [identityBlock([256,256,1024],[1,3,1]) for i in range(5)]

    self.cn4 = convolutionalBlock([512, 512, 2048,2048],[1,3,1,1],[2,1,1,2])
    self.id4 = [identityBlock([256,256,2048],[1,3,1]) for i in range(2)]
    self.averagePooling = AveragePooling2D(padding='same')
    self.f = Flatten()
    self.dense = Dense(numberOfClasses, activation='softmax')

  def call(self,x):
    x = self.zeroPadding(x)
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.activation(x)
    x = self.maxPool1(x)

    x = self.cn1(x)
    for id in self.id1:
      x = id(x)

    x = self.cn2(x)
    for id in self.id2:
      x = id(x)

    x = self.cn3(x)
    for id in self.id3:
      x = id(x)

    x = self.cn4(x)
    for id in self.id4:
      x = id(x)
    
    x = self.averagePooling(x)
    x = self.f(x)
    x = self.dense(x)

    return x


In [31]:
model = resNet50(10)

In [32]:
model.build((None,64,64,3))

In [33]:
model.summary()

Model: "res_net50_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_7 (ZeroPaddin multiple                  0         
_________________________________________________________________
conv2d_281 (Conv2D)          multiple                  9472      
_________________________________________________________________
batch_normalization_273 (Bat multiple                  256       
_________________________________________________________________
activation_83 (Activation)   multiple                  0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 multiple                  0         
_________________________________________________________________
convolutional_block_20 (conv multiple                  76928     
_________________________________________________________________
identity_block_62 (identityB multiple                  